# Day2 - Gradio!

In [ ]:
import os
os.chdir('..')

from utils.utils import load_env_variables
from agents.agents_v2 import ClaudeAgent, OpenAIAgent
from scraper.scraper import SoupScraper

In [ ]:
import gradio as gr

In [ ]:
load_env_variables()

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
def message_gpt(prompt: str):
    agent = OpenAIAgent()
    return agent.respond(system_message, prompt)

In [ ]:
def stream_gpt(prompt: str):
    agent = OpenAIAgent()
    response = agent.respond_with_stream(system_message, prompt)
    for message in response:
        yield message

In [ ]:
message_gpt("What is today's date?")

## User Interface!

In [ ]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(
    fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never"
).launch(inbrowser=True)

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""
gr.Interface(
    fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode
).launch()

In [ ]:
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_claude(prompt: str):
    agent = ClaudeAgent()
    response = agent.respond_with_stream(system_message, prompt)
    for message in response:
        yield message

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
)
view.launch()

## Create Company Brochure

In [ ]:
system_message = (
    "You are an assistant that analyzes the contents of "
    "a company website landing page and creates a short "
    "brochure about the company for prospective customers, "
    "investors and recruits. Respond in markdown."
)

In [ ]:
def stream_brochure(company_name: str, url: str, model: str):
    scraper = SoupScraper(url=url)
    prompt = (
        f"Please generate a company brochure for {company_name}. "
        f"Here is their landing page:\n{scraper.get_contents()}"
    )
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name:"),
        gr.Textbox(label="Company URL:"),
        gr.Dropdown(["GPT", "Claude"], label="Select Model"),
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()